In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

data = pd.read_csv('./data/train.csv')
data.head()

new_branch_id=[2,4,6,12,22,23,24,25,26,27,28,30,31,32,33,37]
old_branch_id=list(set(data.branch_id.unique())-set(new_branch_id))
data['application_date']=pd.to_datetime(data['application_date'])
data['branch_id_new']=data.loc[data.branch_id.isnull(),'state'].map({'TRIPURA':2,
                                   'TAMIL NADU':4,
                                   'PUNJAB':6,
                                   'ASSAM':12,
                                   'UTTAR PRADESH':22,
                                   'MAHARASHTRA':23,
                                   'ORISSA':24,
                                   'KARNATAKA':25,
                                   'KERALA':26,
                                   'MADHYA PRADESH':27,
                                   'HARYANA':28,
                                   'BIHAR':30,
                                   'GUJARAT':31,
                                   'CHHATTISGARH':32,
                                   'WEST BENGAL':33,
                                   'JHARKHAND':37})
data.loc[data.branch_id.isnull(),'branch_id']= data['branch_id_new']
data=data.drop('branch_id_new',axis=1)

d1 = data['application_date'].min()
d2 = pd.to_datetime(data.groupby('branch_id')['application_date'].max().unique())

# this will give you a list containing all of the dates
dd_newlist = [d1 + timedelta(days=x) for x in range((d2[1]-d1).days + 1)]
dd_oldlist = [d1 + timedelta(days=x) for x in range((d2[0]-d1).days + 1)]

new=pd.DataFrame({'branch_id':np.repeat(old_branch_id,len(dd_oldlist)),
             'application_date':dd_oldlist*len(old_branch_id)}).dropna()
new = new.append(pd.DataFrame({'branch_id':np.repeat(new_branch_id,len(dd_newlist)),
             'application_date':dd_newlist*len(new_branch_id)}), ignore_index=True)

branch=data[['branch_id','segment','state']].drop_duplicates().copy()
state=data[['state','zone']].drop_duplicates().dropna().copy()
state=state[~((state.state == 'ORISSA') & (state.zone == 'SOUTH'))]
branch=pd.merge(branch,state,on='state',how='left')

new=pd.merge(new,branch,on='branch_id',how='left')

data=data.drop(['state','zone'],axis=1)
data=pd.merge(new,data,on=['branch_id','segment','application_date'],how='left')
data.head()
data.to_csv('preprocessed_train_for_tbats.csv',index=False)